# Himawari download and plot code (Claude.ai credit)

Timespan Start: 2024/11/03 12:00 - 2024/11/15 16:40 from [CIRA loop here](https://satlib.cira.colostate.edu/weather_media/parade-of-typhoons-in-the-western-pacific/) , the first 4-active-typhoon scene in recorded history occurs on [Nov 11 00Z](https://en.wikipedia.org/wiki/Typhoon_Man-yi#/media/File:Yinxing,_Toraji,_Usagi,_and_Man-yi_2024-11-11_0000Z.jpg), Man-Yi hit Plillippines and dissipated Nov 20. 

In [1]:
# Himawari-9 Band 13 Data Visualization, Downloads and processes AHI-L1b-FLDK data from AWS S3

# Install required packages (run once)
# !pip install satpy[all] s3fs matplotlib numpy cartopy

import s3fs
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, to_rgba
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from satpy import Scene
from datetime import datetime, timedelta
import tempfile
import os
from pathlib import Path

# suppress a warning in output 
import warnings
warnings.filterwarnings("ignore", message="invalid value encountered in log")

In [ ]:
# Configuration of download, and DO IT 
# START_DATE = datetime(2024, 11, 3, 12, 0) # 11-3 to 11-20 is the parade, 11-17-08 is closing of last eye

START_DATE = datetime(2024, 11, 8, 6, 0) # Need full disk from here, in the zoom-out sequence 
END_DATE = datetime(2024, 11, 17, 8, 0) # Need full-disk here, overwrite my top-half-only images 

#END_DATE = datetime(2024, 11, 22, 0, 0)   # Adjust as needed, full length here, just keep restarting! 
#END_DATE = datetime(2024, 11, 20, 0, 0)   # A good endpoint is 11/20 at 00Z  

TIME_INTERVAL = 10  # minutes
BAND = 13
SEGMENTS = range(1, 11)  # Segments 1-6 is 1,7. Need full disk all the way from himawari9_b13_20241108_0600
TOTAL_SEGMENTS = 10
RESOLUTION = "R20"
OUTPUT_DIR = "/Volumes/SamsungUSB/Himawari_parade_fulldisk"

# Custom colormap with transparency
color_points = [
    (190, "#dc05ef", 1.0),  # magenta 
    (222, "#0589ef", 1.0),  # blue
    (240, "#00ffff", 1.0),  # cyan
    (250, "#716f6f", 1.0),  # darker gray 
    (270, "#c5c6c6", 1.0),  # light-mid gray
    (280, "#ffffff", 0.7),  # white semitrans
    (290, "#ffffff", 0.3),  # white point surface
    (300, "#ffffff", 0.1),  # SST point water
    (305, "#ffffff", 0.3),  # white 
    (310, "#ff8000", 0.6),  # hot orange
    (340, "#000000", 0.8)   # very hot black
]
vmin, vmax = 190, 340
custom_cmap = LinearSegmentedColormap.from_list("custom", [
    ((v - vmin) / (vmax - vmin), to_rgba(c, a)) for v, c, a in color_points
])

def generate_urls(dt):
    """Generate S3 URLs for all segments at given datetime"""
    base = f"AHI-L1b-FLDK/{dt:%Y/%m/%d/%H%M}"
    return [
        f"noaa-himawari9/{base}/HS_H09_{dt:%Y%m%d_%H%M}_B{BAND:02d}_FLDK_{RESOLUTION}_S{seg:02d}{TOTAL_SEGMENTS}.DAT.bz2"
        for seg in SEGMENTS
    ]

def get_output_filename(dt, output_dir):
    """Generate output filename for a given datetime"""
    return f"{output_dir}/himawari9_b{BAND:02d}_{dt:%Y%m%d_%H%M}.png"

def already_exists(dt, output_dir):
    """Check if output file already exists"""
    return os.path.exists(get_output_filename(dt, output_dir))

def download_and_plot(dt, output_dir):
    """Download segments and create composite image"""
    fs = s3fs.S3FileSystem(anon=True)
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    with tempfile.TemporaryDirectory() as tmpdir:
        local_files = []
        urls = generate_urls(dt)
        
        # Download segments
        errors = []
        for url in urls:
            try:
                local_path = f"{tmpdir}/{Path(url).name}"
                fs.get(url, local_path)
                local_files.append(local_path)
            except Exception as e:
                errors.append(str(e))
        
        # Need at least some segments
        if len(local_files) < 3:
            return None
        
        # Read with satpy
        scn = Scene(reader='ahi_hsd', filenames=local_files)
        scn.load([f'B{BAND:02d}'])


        # Get the coordinate reference system
        dataset_key = f'B{BAND:02d}'
        crs = scn[dataset_key].attrs['area'].to_cartopy_crs()
        
        plt.figure(figsize=(16, 16), dpi=150)  # Larger size, higher resolution
        
        ax = plt.axes(projection=crs)
        ax.set_facecolor('black')
        ax.add_feature(cfeature.OCEAN, facecolor='black')
        ax.add_feature(cfeature.LAND.with_scale('50m'), facecolor='#587A4E')  
        
        # Add background (e.g., natural Earth or stock image) -- CRASHES 
        # ax.stock_img()  # or ax.add_image(...) with specific tiles
        
        # Overlay satellite data
        ax.imshow(scn[dataset_key], transform=crs, cmap=custom_cmap, vmin=vmin, vmax=vmax, \
                  extent=crs.bounds, origin='upper', zorder=5)
        
        # Add coastlines if needed
        ax.coastlines(color='black')
        #ax.coastlines(resolution='10m', color='white')
        # ax.gridlines()

        ax.set_title(f'Himawari-9 Band {BAND} - {dt:%Y-%m-%d %H:%M} UTC',
                    fontsize=14, fontweight='bold', color='white')
        
        # # Colorbar
        # cbar = plt.colorbar(im, ax=ax, orientation='horizontal', 
        #                    pad=0.05, shrink=0.6)
        # cbar.set_label('Brightness Temperature (K)', fontsize=10)
        # cbar.ax.tick_params(labelsize=9)
        
        # Save
        outfile = get_output_filename(dt, output_dir)
        plt.savefig(outfile, dpi=150, bbox_inches='tight', facecolor='black')
        plt.close()
        
        return outfile

# Main processing loop
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

current = START_DATE
success_count = 0
skip_count = 0
existing_count = 0

print("Starting Himawari-9 image generation...")

while current <= END_DATE:
    if already_exists(current, OUTPUT_DIR):
        existing_count += 1
        current += timedelta(minutes=TIME_INTERVAL)
        continue  # COMMENTING OUT- DO IT ANYWAY & OVERWRITE EXISTING IMAGE FILE
    
    print(f"Processing {current:%Y-%m-%d %H:%M}...", end=" ")
    
    result = download_and_plot(current, OUTPUT_DIR)
    if result:
        print(f"✓ Saved")
        success_count += 1
    else:
        print("✗ Skipped (data unavailable)")
        skip_count += 1
    
    current += timedelta(minutes=TIME_INTERVAL)

print(f"\nCompleted: {success_count} new, {existing_count} existing, {skip_count} skipped")

In [2]:
i,f

(698,
 PosixPath('/Volumes/SamsungUSB/Himawari_parade/himawari9_b13_20241108_1040.png'))

# Define a zooming-out crop process to make the images for ffmpeg

In [10]:
# Cropping holding 1.4 to 1 aspect ratio for as long as possible 
from pathlib import Path
from PIL import Image

SOURCE = Path('/Volumes/SamsungUSB/Himawari_parade/')
OUTPUT = Path('/Volumes/SamsungUSB/Himawari_parade_crop/')
OUTPUT.mkdir(exist_ok=True)

CROP1 = (500, 400, 1200, 850)
CROP2 = (0, 40, 1872, 1912)
TARGET_ASPECT = 1.4  # 14:10 ratio

files = sorted(SOURCE.glob('*.png'))
n = len(files)

for i, f in enumerate(files):
    #if (OUTPUT / f"{i:08d}.png").exists(): continue
    
    img = Image.open(f)
    
    # Interpolate crop box
    t = i / (n - 1)
    left   = CROP1[0] + (CROP2[0] - CROP1[0]) * t
    top    = CROP1[1] + (CROP2[1] - CROP1[1]) * t
    right  = CROP1[2] + (CROP2[2] - CROP1[2]) * t
    bottom = CROP1[3] + (CROP2[3] - CROP1[3]) * t
    
    height = bottom - top
    width = height * TARGET_ASPECT  # Force 1.4:1 aspect by expanding width
    
    # Center horizontally
    center_x = (left + right) / 2
    left = center_x - width / 2
    right = center_x + width / 2
    
    # Clamp to image bounds
    left = max(0, left)
    right = min(img.width, right)
    
    crop = (int(left), int(top), int(right), int(bottom))
    img.crop(crop).save(OUTPUT / f"{i:08d}.png")

print(f"Cropped {n} images to 1.4:1 aspect ratio")

Cropped 2621 images to 1.4:1 aspect ratio


In [11]:
# Resize by upsampling, and pad, always 1000 pixels tall
from PIL import Image
from pathlib import Path

INPUT = Path('/Volumes/SamsungUSB/Himawari_parade_crop/')
OUTPUT = Path('/Volumes/SamsungUSB/Himawari_parade_crop_commonsize/')
OUTPUT.mkdir(exist_ok=True)

TARGET = (1400, 1000)

for filename in sorted(INPUT.glob("*.png")):
    img = Image.open(filename)
    
    # Scale to fill height (1000px), preserve aspect ratio
    aspect = img.width / img.height
    new_height = TARGET[1]
    new_width = int(new_height * aspect)
    img = img.resize((new_width, new_height), Image.LANCZOS)
    
    # Center on black canvas (black bars on sides if width < 1400)
    canvas = Image.new("RGB", TARGET, "black")
    x = (TARGET[0] - img.width) // 2
    canvas.paste(img, (x, 0))
    
    canvas.save(OUTPUT / filename.name)

print(f"Resized to fill {TARGET[1]}px height with side padding as needed")

Resized to fill 1000px height with side padding as needed


In [12]:
filename

PosixPath('/Volumes/SamsungUSB/Himawari_parade_crop/00002620.png')

In [13]:
# RIFE interpolate for pixel jitter 
# Cell 4: RIFE frame interpolation (requires rife-ncnn-vulkan or similar)
from subprocess import run
from pathlib import Path

INPUT_DIR = Path('/Volumes/SamsungUSB/Himawari_parade_crop_commonsize/')
RIFE_DIR = Path('/Volumes/SamsungUSB/Himawari_parade_rife/')

# Clear output directory
for f in RIFE_DIR.glob('*.png'):
    f.unlink()

# Use 2x interpolation (default, should work)
cmd = [
    "/Volumes/SamsungUSB/rife-ncnn-vulkan-20221029-macos/rife-ncnn-vulkan",
    "-i", str(INPUT_DIR),
    "-o", str(RIFE_DIR)
    # No -n flag = 2x interpolation by default
]

print("Running RIFE with 2x interpolation...")
result = run(cmd, capture_output=True, text=True)
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

output_files = sorted(RIFE_DIR.glob('*.png'))
print(f"\nCreated {len(output_files)} output files")
print(f"Expected: ~{2622 * 2 - 1} files")

Running RIFE with 2x interpolation...
STDOUT: 
STDERR: [0 Apple M2]  queueC=0[1]  queueG=0[1]  queueT=0[1]
[0 Apple M2]  bugsbn1=0  bugbilz=0  bugcopc=0  bugihfa=0
[0 Apple M2]  fp16-p/s/a=1/1/1  int8-p/s/a=1/1/1
[0 Apple M2]  subgroup=32  basic=1  vote=1  ballot=1  shuffle=1


Created 5244 output files
Expected: ~5243 files


In [10]:
# TEST Increase keyframe rate to 1s from mp4 video: smaller, still look ok? 

from subprocess import run
from pathlib import Path

# Define input and output files
INPUT_MP4 = '/Users/bmapes/Box/Sky_Symphony_Box/Himawari_parade_zoomout.mp4'
OUTPUT_MP4 = '/Users/bmapes/Box/Sky_Symphony_Box/Himawari_parade_zoomout_1skeys_frommp4.mp4'

# Re-encode with more frequent keyframes
cmd = [
    "ffmpeg", "-y",
    "-i", INPUT_MP4,
    "-c:v", "libx264",
    "-g", "16",  # Keyframe every 16 frames (~1 second at typical framerates)
    "-pix_fmt", "yuv420p",
    OUTPUT_MP4
]

print(f"Re-encoding {INPUT_MP4} with increased keyframe rate...")
run(cmd)
print(f"Saved to {OUTPUT_MP4}")

Re-encoding /Users/bmapes/Box/Sky_Symphony_Box/Himawari_parade_zoomout.mp4 with increased keyframe rate...


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with clang version 18.1.8
  configuration: --prefix=/Users/bmapes/.local/share/mamba/envs/hk25 --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1746479731466/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --disable-gnutls --enable-libvpx --enable-libass --enable-pthreads --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libmp3lame --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-static --enable-version3 --enable-zli

Saved to /Users/bmapes/Box/Sky_Symphony_Box/Himawari_parade_zoomout_1skeys_frommp4.mp4


[out#0/mp4 @ 0x600000448000] video:395391KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.014626%
frame= 5244 fps= 77 q=-1.0 Lsize=  395449KiB time=00:05:27.62 bitrate=9887.9kbits/s speed=4.81x    
[libx264 @ 0x109e04b50] frame I:331   Avg QP:20.15  size:301247
[libx264 @ 0x109e04b50] frame P:3130  Avg QP:24.68  size: 96020
[libx264 @ 0x109e04b50] frame B:1783  Avg QP:26.08  size:  2594
[libx264 @ 0x109e04b50] consecutive B-frames: 32.5% 66.6%  0.2%  0.8%
[libx264 @ 0x109e04b50] mb I  I16..4:  7.1% 53.1% 39.8%
[libx264 @ 0x109e04b50] mb P  I16..4:  0.2%  0.5%  0.3%  P16..4: 29.7% 28.5% 21.9%  0.0%  0.0%    skip:18.9%
[libx264 @ 0x109e04b50] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 20.8%  1.0%  0.2%  direct: 0.5%  skip:77.4%  L0: 8.1% L1:42.3% BI:49.6%
[libx264 @ 0x109e04b50] 8x8 transform intra:52.7% inter:43.3%
[libx264 @ 0x109e04b50] coded y,uvDC,uvAC intra: 81.1% 49.2% 39.4% inter: 36.1% 18.6% 15.8%
[libx264 @ 0x109e04b50] i16 v,h,dc,p: 69% 14% 

In [5]:
OUTPUT_DIR

PosixPath('/Volumes/SamsungUSB/Himawari_parade_rife')

In [15]:
!open /Volumes/SamsungUSB/Himawari_parade_zoomout.mp4

In [ ]:
ffmpeg -i input.mp4 -c:v libx264 -g 60 -keyint_min 60 -sc_threshold 0 -c:a copy output.mp4

In [ ]:
Image.open(f).size